In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
heart_failure =pd.read_csv("/kaggle/input/heartfailure/heartfailure6.csv")

In [ ]:
heart_failure.info()
heart_failure.head()

In [ ]:
###check for missing data
missing_values = heart_failure.isnull().sum()
print(missing_values)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
missing_counts = heart_failure.isnull().sum()

# Plotting the histogram of missing data
plt.figure(figsize=(10, 6))
sns.barplot(x=missing_counts.index, y=missing_counts.values, palette='viridis')
plt.xticks(rotation=45)
plt.xlabel('Columns')
plt.ylabel('Number of Missing Values')
plt.title('Histogram of Missing Data')
plt.show()

Binary variables: anaemia, diabetes, high_blood_pressure, sex, smoking and death_event

In [ ]:
#Drop missing data for binary variables
binary_columns = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking', 'DEATH_EVENT']
binary_df = heart_failure[binary_columns]
binary_df_cleaned = binary_df.dropna()
# Join the cleaned binary columns back to the original DataFrame
cleaned_df = heart_failure.loc[binary_df_cleaned.index]
cleaned_df.info() 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Calculate the number of missing values per column
missing_counts = cleaned_df.isnull().sum()

# Plotting the histogram of missing data
plt.figure(figsize=(10, 6))
sns.barplot(x=missing_counts.index, y=missing_counts.values, palette='viridis')
plt.xticks(rotation=45)
plt.xlabel('Columns')
plt.ylabel('Number of Missing Values')
plt.title('Histogram of Missing Data')
plt.show()

Only age,creatinine_phosphokinase and ejection_fraction have missing values. 
DO mean imputation for these numeral values

In [ ]:
# List of columns for mean imputation
columns_to_impute = ['age', 'creatinine_phosphokinase', 'ejection_fraction']

for col in columns_to_impute:
    mean_value = cleaned_df[col].mean()  # Calculate mean of the column
    cleaned_df[col].fillna(mean_value, inplace=True)  # Fill missing values with the mean
cleaned_df.info()

In [ ]:
##Check assumptions
# Display summary statistics for numerical columns
cleaned_df.describe()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot histograms of numerical features
numerical_cols = cleaned_df.select_dtypes(include='number').columns.tolist()

plt.figure(figsize=(12, 8))
for i, col in enumerate(numerical_cols, start=1):
    plt.subplot(3, 3, i)
    sns.histplot(heart_failure[col], kde=True)
    plt.title(col)
    plt.xlabel("")
plt.tight_layout()
plt.show()


In [ ]:
cleaned_df.head()

In [ ]:
#constant variance
numerical_cols = ['age','creatinine_phosphokinase','ejection_fraction','platelets','serum_sodium']
# Create boxplots for each numerical column
plt.figure(figsize=(12, 8))
sns.boxplot(data=cleaned_df[numerical_cols])
plt.title('Boxplots of Numerical Features')
plt.show()


In [ ]:
X= cleaned_df.iloc[:,:-1].values
y = cleaned_df.iloc[:,-1].values
#Splitting the dataset into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)


In [ ]:
#Do scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
columns_to_scale = ['age', 'creatinine_phosphokinase', 'ejection_fraction','platelets', 'serum_creatinine', 'serum_sodium']


# Fit and transform the scaler on the training data (only on specified columns)
X_train_scaled = X_train.copy()  # copy of X_train
X_test_scaled = X_test.copy()  # copy of X_test

# Apply scaling only to the specified columns
X_train_scaled[:, [cleaned_df.columns.get_loc(col) for col in columns_to_scale]] = scaler.fit_transform(X_train[:, [cleaned_df.columns.get_loc(col) for col in columns_to_scale]])
X_test_scaled[:, [cleaned_df.columns.get_loc(col) for col in columns_to_scale]] = scaler.transform(X_test[:, [cleaned_df.columns.get_loc(col) for col in columns_to_scale]])


In [ ]:
###KNN 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski',p=2)

# Fit KNN model on scaled training data
classifier.fit(X_train_scaled, y_train)


In [ ]:
# Predict on scaled testing data
y_pred_knn = classifier.predict(X_test_scaled)
# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred_knn)
print("Accuracy:", accuracy)

# Generate classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_knn))

# Generate confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_knn))


In [ ]:
#AUC and ROC curves
from sklearn.metrics import roc_curve, roc_auc_score
# Predict probabilities of positive class (class 1)
y_probabilities = classifier.predict_proba(X_test_scaled)[:, 1]

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_probabilities)
# Calculate AUC score
auc_score_knn = roc_auc_score(y_test, y_probabilities)
print("AUC Score:", auc_score_knn)
# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='b', label='ROC Curve (AUC = {:.2f})'.format(auc_score_knn))
plt.plot([0, 1], [0, 1], color='r', linestyle='--', label='Random Guessing')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()



In [ ]:
##SVM
from sklearn.svm import SVC

svm = SVC(kernel='rbf',probability=True, random_state=0)  # 'rbf' kernel is commonly used for non-linear separation
svm.fit(X_train_scaled, y_train)


In [ ]:
# Predict on the test set
y_pred_svm = svm.predict(X_test_scaled)

In [ ]:
# Calculate accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("Accuracy:", accuracy_svm)

# Generate classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_svm))

# Generate confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_svm))


In [ ]:
#ROC AND AUC
# Predict probabilities for the positive class (class 1)
y_probabilities = svm.predict_proba(X_test_scaled)[:, 1]

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_probabilities)

# Calculate AUC score
auc_score_svm = roc_auc_score(y_test, y_probabilities)
print("AUC Score:", auc_score_svm)
# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='b', label='ROC Curve (AUC = {:.2f})'.format(auc_score_svm))
plt.plot([0, 1], [0, 1], color='r', linestyle='--', label='Random Guessing')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()


In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)

rf_classifier.fit(X_train_scaled, y_train)
#rf_classifier.fit(X_train_scaled, y_train)


In [ ]:
# Predict on the test set
y_pred_rf = rf_classifier.predict(X_test_scaled)

In [ ]:
# Calculate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Accuracy:", accuracy_rf)

# Generate classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_rf))

# Generate confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))


In [ ]:
# Predict probabilities for the positive class (class 1)
y_probabilities = rf_classifier.predict_proba(X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test, y_probabilities)

# Calculate AUC score
auc_score_rf = roc_auc_score(y_test, y_probabilities)
print("AUC Score:", auc_score_rf)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='b', label='ROC Curve (AUC = {:.2f})'.format(auc_score_rf))
plt.plot([0, 1], [0, 1], color='r', linestyle='--', label='Random Guessing')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(criterion = 'entropy',random_state=0)
dt_classifier.fit(X_train_scaled, y_train)

In [ ]:
# Predict on the test set
y_pred_dt = dt_classifier.predict(X_test_scaled)

In [ ]:
# Calculate accuracy
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Accuracy:", accuracy_dt)

# Generate classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_dt))

# Generate confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_dt))

In [ ]:
# Predict probabilities for the positive class (class 1)
y_probabilities = dt_classifier.predict_proba(X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test, y_probabilities)

# Calculate AUC score
dt_auc_score = roc_auc_score(y_test, y_probabilities)
print("AUC Score:", dt_auc_score)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='b', label='ROC Curve (AUC = {:.2f})'.format(dt_auc_score))
plt.plot([0, 1], [0, 1], color='r', linestyle='--', label='Random Guessing')
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()


In [ ]:
# Calculate error rates for each classifier
error_rate_knn = 1 - accuracy
error_rate_svm = 1 - accuracy_svm
error_rate_rf = 1 - accuracy_rf
error_rate_dt = 1 - accuracy_dt

In [ ]:
import pandas as pd

# Define the data for the table
data = {
    'Classifier': ['KNN', 'SVM', 'Random Forest', 'Decision Tree'],
    'AUC': [auc_score_knn, auc_score_svm, auc_score_rf, dt_auc_score],
    'Accuracy': [accuracy, accuracy_svm, accuracy_rf, accuracy_dt],
    'Error Rate': [error_rate_knn, error_rate_svm, error_rate_rf, error_rate_dt]
}

# Create DataFrame
df_comparison = pd.DataFrame(data)

# Display the DataFrame
print(df_comparison)
